# Code to generate heretostructures from Jarvis

In [1]:
from jarvis.db.figshare import data
import os
from jarvis.io.vasp.inputs import Poscar
from jarvis.core.atoms import Atoms
#from jarvis.analysis.interface.zur import make_interface
from jarvis_interface import make_interface
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from jarvis.core.atoms import pmg_to_atoms
import Elastic
import pandas as pd
import numpy as np
import contextlib
#import pymatgen.core.interface
#from pymatgen.analysis.interfaces import zsl
from pyxtal import pyxtal
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.core.operations import SymmOp
from pymatgen.core.periodic_table import Element
import heterostructure

/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


### Find matching pairs and output symmetric stackings

In [2]:
count =0
#### deatils of material
dft_2d = data('dft_2d')
df0 = pd.DataFrame(data=dft_2d)
original_id1 = []
original_id2 = []
formula_c = []
mismatch = []
mis_count = []
elastic_energy = []
monolayer_atom = []
write_to_folder = '../data/test/'
for n1,mat1 in enumerate(dft_2d):
    for n2,mat2 in enumerate(dft_2d):
        if n2 > n1 and mat1['formula'] != mat2['formula'] \
            and mat1['elastic_tensor'] != 'na' and mat2['elastic_tensor'] != 'na'\
            and ('OPT-Bandst' in [i.split(',', 1)[0] for i in mat1['raw_files']])\
            and ('OPT-Bandst' in [i.split(',', 1)[0] for i in mat2['raw_files']]):
            #and not (if_toxic_radioac(mat1['formula']) or if_toxic_radioac(mat2['formula'])):
        #and mat1['jid'].split('-')[1] == '14459' and mat2['jid'].split('-')[1] == '27774':

            natom1=mat1['nat']
            natom2=mat2['nat']
            subs = Atoms.from_dict(mat1['atoms'])
            film = Atoms.from_dict(mat2['atoms'])
            if not (heterostructure.if_janus(subs) and heterostructure.if_janus(film)):
                try:
                    hetero = make_interface(film=film, subs=subs, seperation = 2.5,vacuum = 15)
                    hetatoms = hetero['interface'].num_atoms
                    # print(mat1['elastic_tensor'])
                    # print(mat2['elastic_tensor'])
                    # print(mat1['raw_files'])
                    # print(mat2['raw_files'])
                    if hetero['mismatch_angle'] < 0.001 and hetatoms == natom1+ natom2 and hetatoms < 11 and abs(hetero['mismatch_u'])<0.01 and abs(hetero['mismatch_v'])<0.01:
                        count +=1
                        original_id1.append(mat1['jid'])
                        original_id2.append(mat2['jid'])
                        mismatch.append([abs(hetero['mismatch_u']),abs(hetero['mismatch_v'])])
                        formula_c.append([mat1['formula'],mat2['formula']])
                        try:
                            #make interface using equilibrium position under elasticity
                            lat1sm = hetero['subs_sl'].lattice_mat
                            lat2sm = hetero['film_sl'].lattice_mat
                            C1 = mat1['elastic_tensor']
                            C2 = mat2['elastic_tensor']
                            ratio_min, ela_energy, Strain_u1, Strain_u2, Strain_v1, Strain_v2 = Elastic.elastic(lat1sm,lat2sm,C1,C2)
                            Strain_1 = [Strain_u1,Strain_u2,0.]
                            Strain_2 = [Strain_v1,Strain_v2,0.]
                            #compute area of the strained unit cell
                            area = np.linalg.norm(np.cross((1 + Strain_u1) * lat1sm[0],(1 + Strain_u2)*lat1sm[1]))
                            ela_en_per_uc = ela_energy * area/1.6 * 1e-1
                            elastic_energy.append(ela_en_per_uc)
                            #apply strain on the lattice and preserve fractional coordinates

                            #make sure fractional coordinates are not changed, and cartesian coordinates vary accordingly
                            film_eq = film.strain_atoms(Strain_2)
                            subs_eq = subs.strain_atoms(Strain_1)
                            film_eq.frac_coords = film.frac_coords
                            subs_eq.frac_coords = subs.frac_coords
                            film_eq.cart_coords = film_eq.lattice.cart_coords(film_eq.frac_coords)
                            subs_eq.cart_coords = subs_eq.lattice.cart_coords(subs_eq.frac_coords)

                            
                            #apply flipping first
                            film_flipped_list = heterostructure.Apply_flipping(film_eq)
                            subs_flipped_list = heterostructure.Apply_flipping(subs_eq)
                            heter_atom = []#store all possible heterostructures

                            for film_temp_flip in film_flipped_list:
                                for subs_temp_flip in subs_flipped_list:
                                    R_film, R_subs = heterostructure.Rot_Center(film_temp_flip,subs_temp_flip,0,0)
                                    film_rot_ctr = film_temp_flip.center_around_origin(R_film)
                                    film_eq_list = heterostructure.Apply_Rotation(film_rot_ctr)
                                    for ind_film, film_temp in enumerate(film_eq_list):
                                        #Compute the displacement of center of mass to make the alignment of corresponding maximal wyckoff positions
                                        #DISPLACEMENT_wyckoff(film,subs,wyckoff_indices_film,wyckoff_indices_substrate) output displacement
                                        #(fractional coordinates with respect to substrate lattice
                                        Displacement = heterostructure.DISPLACEMENT_wyckoff(film_temp,subs_temp_flip,0,0)
                                        #compute translation
                                        t_list = heterostructure.Translation_List(film_temp,subs_temp_flip)
                                        #make heterostructure from translation
                                        het_list_before_check = heterostructure.Make_Het_Translation(film_temp,subs_temp_flip,t_list, Displacement, seperation=3,vacuum=20)
                                        for het_temp in het_list_before_check:
                                            heter_atom.append(het_temp)

                            #check redundency
                            heter_atom1 = heterostructure.exclude_duplicate(heter_atom)
                            #fine-selection of vdW gap
                            heter_vdw = heterostructure.gap_adjust(heter_atom1)
                            #save heterostructure to Poscar file
                            for i,het in enumerate(heter_vdw):
                                het_atom = het['interface'].center_around_origin()
                                layers = [mat1['jid'].split('-')[1],mat2['jid'].split('-')[1]]
                                names = [mat1['formula'],mat2['formula']]
                                ind_2 = chr(97 + i % 5)
                                ind_1 = str(int(i/5))

                                Poscar(het_atom).write_file(write_to_folder + 'POSCAR_'+
                                                        names[0]+'_'+names[1]+'_'+ layers[0]+'_'+layers[1]+'_'+ ind_1 + '_' + ind_2 + '.vasp')
                            #Poscar(subs).write_file('/Users/user/Documents/pycode/Lattice matching/monolayer_poscar/POSCAR_'+names[0]+'_'+layers[0]+'.vasp')
                            #Poscar(film).write_file('/Users/user/Documents/pycode/Lattice matching/monolayer_poscar/POSCAR_'+names[1]+'_'+layers[1]+'.vasp')
                            #if np.linalg.norm(hetlatsm1[0] - subs.lattice_mat[0]) > 1e-1 * np.linalg.norm(hetlatsm1[0]) or\
                                #np.linalg.norm(hetlatsm1[1] - subs.lattice_mat[1]) > 1e-1 * np.linalg.norm(hetlatsm1[1]):
                                #mis_count.append([names,count])
                                #monolayer_atom.append([subs,film])
                                #heter_atom.append(het_eq)
                            #elif np.linalg.norm(hetlatsm2[0] - film.lattice_mat[0]) > 1e-1 * np.linalg.norm(hetlatsm2[0]) or\
                                #np.linalg.norm(hetlatsm2[1] - film.lattice_mat[1]) > 1e-1 * np.linalg.norm(hetlatsm2[1]):
                                #mis_count.append([names,count])
                                #monolayer_atom.append([subs,film])
                                #heter_atom.append(het_eq)
                        except Exception as e:
                            print('error',e)
                            #elastic_energy.append(999)
                            #pass
                except:
                    pass

d = {'original_id1':original_id1,'original_id2':original_id2,'formula':formula_c,'mismatch':mismatch,'elastic_energy':elastic_energy}
mat_df = pd.DataFrame(data=d)

Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.


/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/opt/anaconda3/envs/newenv/lib/python3.12/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/opt/anaconda3/envs/newenv/lib/python

In [ ]:
#order the mat_df with respect to mismatch
mat_sorted = mat_df.sort_values(by='elastic_energy')
mat_new = mat_sorted[['original_id1','original_id2','formula','mismatch','elastic_energy']]

# Define the file path and name
file_path = './jarvis_initial.xlsx'

# Output the DataFrame to an Excel file
mat_new.to_excel(file_path, index=False)

print(f"DataFrame has been written to {file_path}")

DataFrame has been written to /Users/user/Documents/pycode/Lattice matching/jarvis_initial.xlsx


In [ ]:
#remove duplicate materials
file_path = './jarvis_initial.xlsx'
mat_new = pd.read_excel(file_path)
seen = []
indices_to_keep = []
for i in range(len(mat_new)):
    match = mat_new.iloc[i]
    if i == 0:
        seen.append(match['formula'])
        indices_to_keep.append(i)
        continue
    else:
        formula = match['formula']
        if formula not in seen:
            seen.append(formula)
            indices_to_keep.append(i)

filtered_df = mat_new.iloc[indices_to_keep]
new_path = './jarvis_170125.xlsx'
filtered_df.to_excel(new_path, index=False)
print(f"DataFrame has been written to {new_path}")

DataFrame has been written to /Users/user/Documents/pycode/Lattice matching/jarvis_initial.xlsx


# compare structure with C2DB using pymatgen

In [3]:
import pandas as pd
import numpy as np
import ast
from pymatgen.core.structure import Structure
#from ase import Atoms
from ase.io import read
from pymatgen.io.ase import AseAtomsAdaptor
from jarvis.core.atoms import pmg_to_atoms
from pymatgen.analysis.structure_matcher import StructureMatcher
import os
#input matched pairs
file_path = '../data/matches_output/matches_output_jarvis.xlsx'
mat_df_jar = pd.read_excel(file_path) 
file_path = '../data/matches_output/pairs_c2db_new_1to1_Sep.xlsx'
mat_df_c2 = pd.read_excel(file_path) 
#search folders containing name_part from formula
def search_folders(search_path, name_part):
    # List to store matching folder paths
    matching_folders = []
    
    # Walk through the directory
    for root, dirs, files in os.walk(search_path):
        for dir_name in dirs:  # Iterate over directories instead of files
            if name_part in dir_name:  # Check if part of the name is in the directory name
                matching_folders.append(os.path.join(root, dir_name))
    
    return matching_folders
    
from jarvis.core.atoms import Atoms
def remove_vac(atom1):
    top_z = max(np.array(atom1.cart_coords)[:, 2])
    bot_z = min(np.array(atom1.cart_coords)[:, 2])
    lat_z = [0.,0.,top_z - bot_z]
    lat_mat = [atom1.lattice_mat[0],atom1.lattice_mat[1],lat_z]
    new_atoms_cart = Atoms(lattice_mat=lat_mat, coords=atom1.cart_coords, elements=atom1.elements, cartesian=True)
    #print(new_atoms)
    frac_coords = new_atoms_cart.frac_coords
    translation = [0.,0.,round(frac_coords[0,2])]
    frac_coords[:] = frac_coords[:] - translation
    atom1_n = Atoms(lattice_mat=lat_mat, coords=frac_coords, elements=atom1.elements, cartesian=False)
    return atom1_n

In [ ]:
count = 0#number of materials found exist in both databases
count_same = 0#number of materials that have same structures
layers = []
names = []
for i in range(len(mat_df_jar)):
    pair = mat_df_jar.iloc[i]
    jid1 = pair['original_id1'].split('-')[1]
    jid2 = pair['original_id2'].split('-')[1]
    if not jid1 in layers:
        layers.append(pair['original_id1'].split('-')[1])
        names.append(ast.literal_eval(pair['formula'])[0])
    if not jid2 in layers:
        layers.append(pair['original_id2'].split('-')[1])
        names.append(ast.literal_eval(pair['formula'])[1])
    
for j in range(len(layers)):
    if_same = 0
    Path_jar = '../data/monolayer_poscar_jarvis/POSCAR_'+names[j]+'_'+layers[j]+'.vasp'
    structure_ja = Structure.from_file(Path_jar)
    atom_ja = Atoms.from_poscar(Path_jar)
    # Search for structure file in c2db
    search_path = '../data/c2db 3'
    name_part = structure_ja.reduced_formula
    paths = search_folders(search_path, name_part)
    # Output the folder paths
    count_temp = 0
    for path in paths:
        if if_same == 1:
            break
        for z in range(5):
            t = z + 1
            path1 = path + "/" + str(t) + "/structure.xyz"
            try:
                structure_c2 = AseAtomsAdaptor.get_structure(read(path1))
                atom_c2 = pmg_to_atoms(structure_c2)
                if name_part == structure_c2.reduced_formula:
                    if count_temp == 0:
                        count += 1
                        count_temp = 1
                    #remove vacuum
                    ja_new = remove_vac(atom_ja)
                    c2db_new = remove_vac(atom_c2)
                    ja_new_py = ja_new.pymatgen_converter()
                    c2db_new_py = c2db_new.pymatgen_converter()
                    matcher = StructureMatcher(ltol = 0.05, stol = 0.05, angle_tol = 2, scale=False)
                    ifsame = matcher.fit(ja_new_py,c2db_new_py)
                    if ifsame:
                        if_same = ifsame
                        print(path1)
                        print(Path_jar)
                        break
            except Exception as error:
                #print(error)
                pass
            
    if if_same == 1:
        count_same += 1
        print(name_part)
print(count)
print(count_same)

/Users/user/Documents/Files/Acad/Lattice coincidence/data/c2db 3/AB2/1NbSe2/1/structure.xyz
/Users/user/Documents/Files/Acad/Lattice coincidence/data/monolayer_poscar_jarvis/POSCAR_NbSe2_655.vasp
NbSe2
/Users/user/Documents/Files/Acad/Lattice coincidence/data/c2db 3/AB2/1MoSe2/1/structure.xyz
/Users/user/Documents/Files/Acad/Lattice coincidence/data/monolayer_poscar_jarvis/POSCAR_MoSe2_649.vasp
MoSe2
/Users/user/Documents/Files/Acad/Lattice coincidence/data/c2db 3/AB2/1WSe2/1/structure.xyz
/Users/user/Documents/Files/Acad/Lattice coincidence/data/monolayer_poscar_jarvis/POSCAR_WSe2_652.vasp
WSe2
/Users/user/Documents/Files/Acad/Lattice coincidence/data/c2db 3/AB2/1CoO2/1/structure.xyz
/Users/user/Documents/Files/Acad/Lattice coincidence/data/monolayer_poscar_jarvis/POSCAR_CoO2_14441.vasp
CoO2
/Users/user/Documents/Files/Acad/Lattice coincidence/data/c2db 3/AB2/1SnSe2/1/structure.xyz
/Users/user/Documents/Files/Acad/Lattice coincidence/data/monolayer_poscar_jarvis/POSCAR_SnSe2_762.vasp


# Check if layer distances in relaxed structures matches with van der Waals radius

In [ ]:
import glob
from pymatgen.core.periodic_table import Element
import os

#input matched pairs

layers = []
names = []
pathlist = glob.glob("../better_relaxed/*.vasp")
Num_het = len(pathlist)
dis_van_average = []
for hetero_path in pathlist:
    file_name = os.path.basename(hetero_path)
    name1 = file_name.split('.')[0].split('_')[1]
    name2 = file_name.split('.')[0].split('_')[2]
    jid1 = file_name.split('.')[0].split('_')[3]
    jid2 = file_name.split('.')[0].split('_')[4]
    try:
        path_mono1 = '../data/monolayer_poscar(jarvis)/POSCAR_'+name1+'_'+jid1+'.vasp'
        path_mono2 = '../data/monolayer_poscar(jarvis)/POSCAR_'+name2+'_'+jid2+'.vasp'
        structure_mono1 = Structure.from_file(path_mono1)
        structure_mono2 = Structure.from_file(path_mono2)
    except:
        path_mono1 = '../data/monolayer_poscar(jarvis)/POSCAR_'+name1+'_'+jid2+'.vasp'
        path_mono2 = '../data/monolayer_poscar(jarvis)/POSCAR_'+name2+'_'+jid1+'.vasp'
        structure_mono1 = Structure.from_file(path_mono1)
        structure_mono2 = Structure.from_file(path_mono2)
    
    structure_het = Structure.from_file(hetero_path)

    #record number of atoms in each layer
    num_subs = len(structure_mono1.species)
    num_film = len(structure_mono2.species)
    #compute relative distance between atoms in two layers
    #simply take every element's distances to other elements and compare with the sum of their van der Waals radius
    species = structure_het.species
    distances = structure_het.distance_matrix
    dis_sort = np.zeros((len(species),len(species)))

    #renormalised (distance - vdW radius)/vdW radius sorted by z coordinates 
    dis_van_norm = np.zeros((len(species),len(species)))
    dis_ind = np.argsort(np.argsort(structure_het.cart_coords[:,2]))
    
    for i, i_sort in enumerate(dis_ind):
        for j, j_sort in enumerate(dis_ind):
            if i != j:
                sum_vdw = species[i].van_der_waals_radius + species[j].van_der_waals_radius
                dis_sort[i_sort, j_sort] = distances[i,j]
                dis_van_norm[i_sort, j_sort] = dis_van[i_sort, j_sort]/sum_vdw
                
    #record distance - vdW radius between subs atoms and film atoms
    dis_van_cross = np.zeros((num_subs,num_film))
    for i in range(num_subs):
        for j in range(num_film):
            dis_van_cross[i,j] = dis_van_norm[i,num_subs + j]
    dis_van_average.append(np.min(dis_van_cross))
print(dis_van_average)
print(pathlist[-1])